# Transforming

In this notebook, we will explore how to use Large Language Models for text transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.

## Setup

In [3]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [4]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    client = openai.OpenAI()  # ✅ Initialize the client
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    return response.choices[0].message.content

## Translation

ChatGPT is trained with sources in many languages. This gives the model the ability to do translation. Here are some examples of how to use this capability.

In [5]:
prompt = f"""
Translate the following English text to hindi: \ 
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)

नमस्ते, मुझे एक ब्लेंडर आर्डर करना है।


In [17]:
prompt = f"""
Tell me which language this is: 
```नमस्ते, मुझे एक ब्लेंडर आर्डर करना है।```
"""
response = get_completion(prompt)
print(response)

This is Hindi.


In [7]:
prompt = f"""
Translate the following  text to French and hindi
and English pirate: \
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)

French: Je veux commander un ballon de basket
Hindi: मुझे एक बास्केटबॉल ऑर्डर करना है
English: I want to order a basketball


In [8]:
prompt = f"""
Translate the following text to hindi in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)

Formal: क्या आप एक तकिया आर्डर करना चाहेंगे?
Informal: क्या तुम एक तकिया आर्डर करना चाहोगे?


### Universal Translator
Imagine you are in charge of IT at a large multinational e-commerce company. Users are messaging you with IT issues in all their native languages. Your staff is from all over the world and speaks only their native languages. You need a universal translator!

In [9]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

In [18]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Hindi: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message (French): La performance du système est plus lente que d'habitude.
English: "The system performance is slower than usual."

Hindi: "सिस्टम का प्रदर्शन सामान्य से धीमा है।" 

Original message (This is Spanish.): Mi monitor tiene píxeles que no se iluminan.
English: My monitor has pixels that do not light up.

Hindi: मेरे मॉनिटर में पिक्सेल हैं जो जलाए नहीं जाते। 

Original message (Italian): Il mio mouse non funziona
English: My mouse is not working
Hindi: मेरा माउस काम नहीं कर रहा है। 

Original message (This is Polish.): Mój klawisz Ctrl jest zepsuty
English: My Ctrl key is broken
Hindi: मेरा कंट्रोल कुंजी टूट गई है। 

Original message (This is Chinese.): 我的屏幕在闪烁
English: My screen is flickering
Hindi: मेरा स्क्रीन फ्लिकर कर रहा है। 



## Tone Transformation
Writing can vary based on the intended audience. ChatGPT can produce different tones.


In [19]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is not ok everytime i work my ass slogging here and everytime get neglected jiust fuck off'
"""
response = get_completion(prompt)
print(response)

Dear Sir/Madam,

I am writing to express my dissatisfaction with the current situation at work. Despite my hard work and dedication, I feel that I am being consistently overlooked and neglected. This is unacceptable and I request that this issue be addressed promptly.

Sincerely,
[Your Name]


## Format Conversion
ChatGPT can translate between formats. The prompt should describe the input and output formats.

In [12]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)

<html>
<head>
  <title>Restaurant Employees</title>
</head>
<body>
  <table>
    <tr>
      <th>Name</th>
      <th>Email</th>
    </tr>
    <tr>
      <td>Shyam</td>
      <td>shyamjaiswal@gmail.com</td>
    </tr>
    <tr>
      <td>Bob</td>
      <td>bob32@gmail.com</td>
    </tr>
    <tr>
      <td>Jai</td>
      <td>jai87@gmail.com</td>
    </tr>
  </table>
</body>
</html>


In [13]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


## Spellcheck/Grammar check.

Here are some examples of common grammar and spelling problems and the LLM's response. 

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.

In [14]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

The girl with the black and white puppies has a ball.
No errors found
No errors found.
Their goes my freedom. There going to bring they’re suitcases.

No errors found

Rewritten:
Their goes my freedom. There going to bring their suitcases.
You're going to need your notebook.
No errors found.
No errors found


In [15]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

Got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.


In [20]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

<span style='color:red;font-weight:700;text-decoration:line-through;'>Got this for </span><span style='color:green;font-weight:700;'>Dear Sir/Madam, </span>

<span style='color:green;font-weight:700;'>I am writing to express </span>my <span style='color:red;font-weight:700;text-decoration:line-through;'>daughter for her birthday cuz she keeps taking mine from </span><span style='color:green;font-weight:700;'>dissatisfaction with the current situation at work. Despite </span>my <span style='color:red;font-weight:700;text-decoration:line-through;'>room.  Yes, adults also like pandas too.  She takes it everywhere with her, </span><span style='color:green;font-weight:700;'>hard work </span>and <span style='color:red;font-weight:700;text-decoration:line-through;'>it's super soft </span><span style='color:green;font-weight:700;'>dedication, I feel that I am being consistently overlooked </span>and <span style='color:red;font-weight:700;text-decoration:line-through;'>cute.  One of the ears </span><span style='color:green;font-weight:700;'>neglected. This </span>is <span style='color:red;font-weight:700;text-decoration:line-through;'>a bit lower than the other, </span><span style='color:green;font-weight:700;'>unacceptable </span>and I <span style='color:red;font-weight:700;text-decoration:line-through;'>don't think </span><span style='color:green;font-weight:700;'>request </span>that <span style='color:red;font-weight:700;text-decoration:line-through;'>was designed to </span><span style='color:green;font-weight:700;'>this issue </span>be <span style='color:red;font-weight:700;text-decoration:line-through;'>asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same price.  It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.</span><span style='color:green;font-weight:700;'>addressed promptly. </span>

<span style='color:green;font-weight:700;'>Sincerely, </span>

<span style='color:green;font-weight:700;'>[Your Name]</span>

In [21]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

I purchased this adorable panda plush for my daughter's birthday as she kept borrowing mine from my room. It's not just kids who love pandas - adults do too! The plush is incredibly soft and cute, and my daughter carries it everywhere with her. However, I did notice that one of the ears is slightly lower than the other, which seems unintentional. Additionally, I found the size to be a bit smaller than expected given the price. I believe there are larger options available for the same cost. Despite this, the plush arrived a day earlier than anticipated, allowing me to enjoy it myself before gifting it to my daughter. Overall, while there are some minor flaws, the quality and cuteness of the panda plush make it a worthwhile purchase.